In [1]:
import sys
print(sys.executable)

c:\Users\48113164\AppData\Local\miniconda3\envs\torch-gpu\python.exe


In [ ]:
import torch
device = torch.device('cuda')

In [ ]:
torch.cuda.is_available()

True

In [8]:
from kp_api import KeypointExtractor

ModuleNotFoundError: No module named 'mmpose'

In [ ]:
import pandas as pd
import subprocess
import os
import numpy as np

# Número de videos a procesar
N = 20

# Leer anotaciones desde CSV
df = pd.read_csv("data/train.csv")  # debe tener youtube_id, start_time, end_time, label

for idx, row in df.iterrows():
    if idx >= N:
        break
    youtube_id = row['youtube_id']
    start = int(row['start_time'])
    end = int(row['end_time'])
    label = row['label']
    
    # Descargar clip usando yt-dlp (del segundo 'start' al 'end')
    url = f"https://youtu.be/{youtube_id}"
    cmd = [
        'yt-dlp', '--quiet', '--no-warnings', 
        '--download-sections', f"*{start}-{end}",
        '-o', 'temp_clip.%(ext)s', url
    ]
    try:
        subprocess.run(cmd, check=True)
    except subprocess.CalledProcessError:
        print(f"No se pudo descargar {youtube_id} {start}-{end}, se omite")
        continue
    
    # Extraer keypoints del clip descargado
    keypoints = KeypointExtractor.extract_from_video("temp_clip.mp4")
    
    # Guardar keypoints en archivo .npy dentro de la carpeta de la etiqueta
    out_dir = os.path.join("output_dir", label)
    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, f"{youtube_id}_{start}_{end}_keypoints.npy")
    np.save(out_path, keypoints)
    
    # Eliminar video temporal
    os.remove("temp_clip.mp4")